<a href="https://colab.research.google.com/github/JohnTaco93/Foxconn-industrial-AI-Data-Challenge/blob/master/PLC_data_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#baseline with the PLC dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import lightgbm as lgb
import datetime
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
import math

import numpy as np
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std


In [ ]:
path_data = "/content/gdrive/My Drive/Foxconn_Challenge_2020/data_foxconn/"

In [ ]:
data01_plc = pd.read_csv(path_data+'/01-Training Data/01/PLC/plc.csv')
data02_plc = pd.read_csv(path_data+'/01-Training Data/02/PLC/plc.csv')
data03_plc = pd.read_csv(path_data+'/01-Training Data/03/PLC/plc.csv')

In [ ]:
def data_preproc(df):
  df['time']=pd.to_datetime(df.time, format = '%H:%M:%S:%f')
  #df['last_time']=df['time'].max() 
  #df['rul']=df['last_time']-df['time']
  #df['rul']=df['rul']/np.timedelta64(1, 'm')

  df['hour']=df.time.dt.hour
  df['minute']=df.time.dt.minute
  #del df['last_time'], df['time']
  del df['time']
  return df

In [ ]:
data01_plc=data_preproc(data01_plc)
data02_plc=data_preproc(data02_plc)
data03_plc=data_preproc(data03_plc)

In [ ]:
data01_plc.head()

,spindle_load,x,y,z,csv_no,hour,minute
0,0.0824,-20.9572,-17.6915,-89.6468,1,11,28
1,0.0824,-20.9572,-17.6915,-89.6468,1,11,28
2,0.0824,-20.9572,-17.6915,-89.6468,1,11,28
3,0.0824,-20.9572,-17.6915,-89.6468,1,11,28
4,0.0824,-20.9572,-17.6915,-89.6468,1,11,28


In [ ]:
data02_plc.head()

,spindle_load,x,y,z,csv_no,hour,minute
0,0.061037,-83.7938,-182.5368,0.0,1,9,42
1,0.061037,-83.7938,-182.5368,0.0,1,9,42
2,0.061037,-83.7938,-182.5368,0.0,1,9,42
3,0.061037,-83.7938,-182.5368,0.0,1,9,42
4,0.061037,-83.7938,-182.5368,0.0,1,9,42


In [ ]:
data03_plc.head()

,spindle_load,x,y,z,csv_no,hour,minute
0,0.045778,-300.9473,127.3867,-277.0397,1,11,42
1,0.045778,-300.9473,127.3867,-277.0397,1,11,42
2,0.045778,-300.9473,127.3867,-277.0397,1,11,42
3,0.045778,-300.9473,127.3867,-277.0397,1,11,42
4,0.045778,-300.9473,127.3867,-277.0397,1,11,42


In [ ]:
#data01_plc=data01_plc.groupby(['hour','minute']).mean().reset_index()
#data02_plc=data02_plc.groupby(['hour','minute']).mean().reset_index()
#data03_plc=data03_plc.groupby(['hour','minute']).mean().reset_index()

data01_plc['id']=1
data02_plc['id']=2
data03_plc['id']=3

data01_plc=data01_plc.groupby(['csv_no']).mean().reset_index()
data02_plc=data02_plc.groupby(['csv_no']).mean().reset_index()
data03_plc=data03_plc.groupby(['csv_no']).mean().reset_index()

data01_plc['rul']=240-(data01_plc['csv_no']-1)*5
data02_plc['rul']=240-(data02_plc['csv_no']-1)*5
data03_plc['rul']=185-(data03_plc['csv_no']-1)*5

In [ ]:
data01_plc.head()

,csv_no,spindle_load,x,y,z,hour,minute,id,rul
0,1,0.914081,-356.155884,44.528058,-208.563484,11,28,1,240
1,2,13.331341,-737.676705,115.590290,-416.320000,11,33,1,235
2,3,13.701669,-737.856319,105.071377,-421.720000,11,49,1,230
3,4,13.355145,-736.889246,114.687300,-423.520000,11,54,1,225
4,5,13.851032,-757.617073,119.884201,-425.320000,11,59,1,220


In [ ]:
data01_plc.tail()

,csv_no,spindle_load,x,y,z,hour,minute,id,rul
43,44,18.551681,-262.163846,122.855433,-427.120000,16,8,1,25
44,45,16.338912,-256.877528,121.644617,-428.349274,16,13,1,20
45,46,18.578114,-261.288195,136.055511,-430.120000,16,18,1,15
46,47,17.217182,-265.601527,104.301247,-431.322719,16,23,1,10
47,48,1.021385,-40.317668,-136.676397,-131.135346,16,28,1,5


In [ ]:
data02_plc.head()

,csv_no,spindle_load,x,y,z,hour,minute,id,rul
0,1,4.425582,-681.708040,49.622495,-280.836206,9,42,2,240
1,2,8.763423,-415.280921,108.637706,-415.882105,9,47,2,235
2,3,10.106236,-585.226604,105.741394,-417.715040,9,52,2,230
3,4,9.955772,-345.678134,199.199593,-418.946896,9,57,2,225
4,5,9.267133,-748.663961,156.516027,-420.239503,10,2,2,220


In [ ]:
data02_plc.tail()

,csv_no,spindle_load,x,y,z,hour,minute,id,rul
43,44,12.552712,-713.164007,144.412558,-480.020507,16,27,2,25
44,45,12.364677,-419.120718,137.487093,-481.602319,16,32,2,20
45,46,11.433770,-698.703146,196.198324,-477.196087,16,37,2,15
46,47,10.774908,-627.187503,186.084878,-472.246480,16,42,2,10
47,48,10.263164,-336.873114,203.164056,-482.745319,16,47,2,5


In [ ]:
data03_plc.head()

,csv_no,spindle_load,x,y,z,hour,minute,id,rul
0,1,1.267259,-316.457348,118.084560,-310.033741,11,42,3,185
1,2,12.595688,-246.128581,118.143456,-531.594600,11,47,3,180
2,3,13.490702,-278.715433,140.684027,-532.951000,11,52,3,175
3,4,12.566447,-253.247554,102.361534,-534.531971,11,57,3,170
4,5,13.651261,-259.052118,145.016604,-535.951000,12,2,3,165


In [ ]:
data03_plc.tail()

,csv_no,spindle_load,x,y,z,hour,minute,id,rul
32,33,11.574848,-859.405446,190.401562,-553.849557,10,37,3,25
33,34,15.227132,-500.552802,48.883475,-496.668167,10,42,3,20
34,35,15.705032,-496.206078,79.907723,-500.569300,10,47,3,15
35,36,16.167322,-502.566138,55.423277,-504.333603,10,52,3,10
36,37,13.488097,-498.994624,12.435645,-505.721843,10,57,3,5


In [ ]:
frames = [data01_plc, data02_plc, data03_plc]

In [ ]:
data_plc = pd.concat(frames)

In [ ]:
import math 
import numpy as np

In [ ]:
def feature_eng(df):
  df['xyz']=np.sqrt(df['x']**2+df['y']**2+df['y']**2)

  return df

In [ ]:
data_plc=feature_eng(data_plc)

In [ ]:
data_plc.drop(['csv_no','hour','minute'],axis=1,inplace=True)

In [ ]:
data_plc.head()

,spindle_load,x,y,z,id,rul,xyz
0,0.914081,-356.155884,44.528058,-208.563484,1,240,361.680121
1,13.331341,-737.676705,115.590290,-416.320000,1,235,755.572069
2,13.701669,-737.856319,105.071377,-421.720000,1,230,752.669872
3,13.355145,-736.889246,114.687300,-423.520000,1,225,754.527743
4,13.851032,-757.617073,119.884201,-425.320000,1,220,776.355635


In [ ]:
data_plc.shape

(133, 7)

In [ ]:
data_plc_vibr=data_plc.copy()

In [ ]:
X=data_plc_vibr.drop(['rul'],axis=1)
y=data_plc_vibr.rul.copy() #target

In [ ]:
ols_model = sm.OLS(y, X)
ols_results = ols_model.fit()
print(ols_results.summary())

                                 OLS Regression Results                                
Dep. Variable:                    rul   R-squared (uncentered):                   0.724
Model:                            OLS   Adj. R-squared (uncentered):              0.711
Method:                 Least Squares   F-statistic:                              55.48
Date:                Wed, 14 Oct 2020   Prob (F-statistic):                    3.65e-33
Time:                        21:10:07   Log-Likelihood:                         -753.22
No. Observations:                 133   AIC:                                      1518.
Df Residuals:                     127   BIC:                                      1536.
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

In [ ]:
#0.724
#0.711

In [ ]:
data01_plc_test = pd.read_csv(path_data+'/02-Testing Data/01/PLC/plc.csv')
data02_plc_test = pd.read_csv(path_data+'/02-Testing Data/02/PLC/plc.csv')
data03_plc_test = pd.read_csv(path_data+'/02-Testing Data/03/PLC/plc.csv')
data04_plc_test = pd.read_csv(path_data+'/02-Testing Data/04/PLC/plc.csv')
data05_plc_test = pd.read_csv(path_data+'/02-Testing Data/05/PLC/plc.csv')

In [ ]:
data01_plc_test=data_preproc(data01_plc_test)
data02_plc_test=data_preproc(data02_plc_test)
data03_plc_test=data_preproc(data03_plc_test)
data04_plc_test=data_preproc(data04_plc_test)
data05_plc_test=data_preproc(data05_plc_test)

In [ ]:
data04_plc_test.head()

,spindle_load,x,y,z,csv_no,hour,minute
0,13.760796,-281.1937,216.7628,-476.463,1,12,23
1,13.760796,-281.1937,216.7628,-476.463,1,12,23
2,13.760796,-281.1937,216.7628,-476.463,1,12,23
3,13.760796,-281.1937,216.7628,-476.463,1,12,23
4,13.760796,-281.1937,216.7628,-476.463,1,12,23


In [ ]:
data01_plc_test['id']=1
data02_plc_test['id']=2
data03_plc_test['id']=3
data04_plc_test['id']=4
data05_plc_test['id']=5

data01_plc_test=data01_plc_test.groupby(['csv_no']).mean().reset_index()
data02_plc_test=data02_plc_test.groupby(['csv_no']).mean().reset_index()
data03_plc_test=data03_plc_test.groupby(['csv_no']).mean().reset_index()
data04_plc_test=data04_plc_test.groupby(['csv_no']).mean().reset_index()
data05_plc_test=data05_plc_test.groupby(['csv_no']).mean().reset_index()

#data01_plc_test['rul']=240-(data01_plc_test['csv_no']-1)*5
#data02_plc_test['rul']=240-(data02_plc_test['csv_no']-1)*5
#data03_plc_test['rul']=185-(data03_plc_test['csv_no']-1)*5

In [ ]:
data01_plc_test.head()

,csv_no,spindle_load,x,y,z,hour,minute,id
0,1,0.056705,-636.564300,46.728800,-380.029000,13,45,1
1,2,14.836691,-241.655016,129.768672,-524.292847,15,10,1
2,3,15.368488,-252.051477,138.808075,-529.952473,15,15,1
3,4,15.361418,-246.874731,132.580058,-535.614881,15,20,1
4,5,13.457013,-236.853588,125.540566,-542.558577,15,25,1


In [ ]:
data04_plc_test.head()

,csv_no,spindle_load,x,y,z,hour,minute,id
0,1,14.864877,-258.843969,137.455786,-476.463000,12,23,4
1,2,13.921689,-267.847211,118.317371,-477.627911,12,28,4
2,3,14.158398,-263.077099,119.842497,-479.422114,12,34,4
3,4,14.400606,-264.219530,122.584868,-481.224927,12,39,4
4,5,14.650881,-256.026264,126.365148,-482.414975,12,44,4


In [ ]:
data01_plc_test=feature_eng(data01_plc_test)
data02_plc_test=feature_eng(data02_plc_test)
data03_plc_test=feature_eng(data03_plc_test)
data04_plc_test=feature_eng(data04_plc_test)
data05_plc_test=feature_eng(data05_plc_test)

In [ ]:
data01_plc_test.head()

,csv_no,spindle_load,x,y,z,hour,minute,id,xyz
0,1,0.056705,-636.564300,46.728800,-380.029000,13,45,1,639.985367
1,2,14.836691,-241.655016,129.768672,-524.292847,15,10,1,303.441861
2,3,15.368488,-252.051477,138.808075,-529.952473,15,15,1,319.476620
3,4,15.361418,-246.874731,132.580058,-535.614881,15,20,1,310.003349
4,5,13.457013,-236.853588,125.540566,-542.558577,15,25,1,296.007584


In [ ]:
data01_plc_test.drop(['csv_no','hour','minute'],axis=1,inplace=True)
data02_plc_test.drop(['csv_no','hour','minute'],axis=1,inplace=True)
data03_plc_test.drop(['csv_no','hour','minute'],axis=1,inplace=True)
data04_plc_test.drop(['csv_no','hour','minute'],axis=1,inplace=True)
data05_plc_test.drop(['csv_no','hour','minute'],axis=1,inplace=True)

In [ ]:
#Preditions

In [ ]:
predictions=[ols_results.predict(data01_plc_test.iloc[-1:]).iloc[0], ols_results.predict(data02_plc_test.iloc[-1:]).iloc[0], 
             ols_results.predict(data03_plc_test.iloc[-1:]).iloc[0], ols_results.predict(data04_plc_test.iloc[-1:]).iloc[0],
             ols_results.predict(data05_plc_test.iloc[-1:]).iloc[0]]

In [ ]:
#submission

In [ ]:
submission=pd.read_csv(path_data+'submission.csv')

In [ ]:
submission

,file,﻿answer
0,files1,100
1,files2,200
2,files3,100
3,files4,200
4,files5,100


In [ ]:
submission['﻿answer']=predictions

In [ ]:
submission

,file,﻿answer
0,files1,79.997289
1,files2,84.643999
2,files3,75.524093
3,files4,65.730820
4,files5,50.738078


In [ ]:
# current position on the leaderboard: 16